~~~
Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
~~~

# Prompt MedGemma 1.5 with DICOM Computed Tomography (CT) Imaging

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/google-health/medgemma/blob/main/notebooks/rl_with_trl.ipynb">
      <img alt="Google Colab logo" src="https://www.tensorflow.org/images/colab_logo_32px.png" width="32px"><br> Run in Google Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogle-Health%2Fmedgemma%2Fmain%2Fnotebooks%2Frl_with_trl.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/google-health/medgemma/blob/main/notebooks/rl_with_trl.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://huggingface.co/collections/google/medgemma-release-680aade845f90bec6a3f60c4">
      <img alt="Hugging Face logo" src="https://huggingface.co/front/assets/huggingface_logo-noborder.svg" width="32px"><br> View on Hugging Face
    </a>
  </td>
</tr></tbody></table>

This notebook demonstrates how 3D representation of [computed tomography (CT)](https://www.nibib.nih.gov/science-education/science-topics/computed-tomography-ct) imaging can be used to prompt MedGemma 1.5 running on Vertex AI. This notebook is for educational purposes only to show the baseline functionality of MedGemma 1.5. It does not represent a finished or approved product, is not intended to diagnose or suggest treatment for any disease or condition, and should not be used for medical advice. See [HAI-DEF Terms of Use](https://developers.google.com/health-ai-developer-foundations/terms) for more information.

Vertex AI makes it easy to serve your model and make it accessible to the world. Learn more about [Vertex AI](https://cloud.google.com/vertex-ai/docs/start/introduction-unified-platform).

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing); use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

In [ ]:
# @title Install pydicom Python library
%%capture
! pip install pydicom

In [ ]:
# @title Authenticate Colab User to connect to DICOM store.
from google.colab import auth


# There will be a popup asking you to sign in with your user account and approve
# access.
auth.authenticate_user()

# Retrieve Imaging Data Commons (IDC) CT Imaging

[Imaging Data Commons (IDC)](ttps://datacommons.cancer.gov/repository/imaging-data-commons#) is one of the largest publicly available, de-identified, repositories for cancer imaging. The repository is funded by the [National Cancer Institute (NCI)](https://www.cancer.gov/), an institute of the [National Institutes of Health (NIH)](https://www.nih.gov/), a part of the [U.S. Department of Health and Human Service](https://www.hhs.gov/). IDC contains imaging for all major medical imaging modalities. Imaging is stored within the archive as [DICOM](https://www.dicomstandard.org/). Imaging and its associated metadata can be searched, visualized through the [IDC website](https://portal.imaging.datacommons.cancer.gov/explore/), [BigQuery](https://cloud.google.com/healthcare-api/docs/resources/public-datasets/idc), and can be accessed using DICOMweb ([IDC tutorial](https://learn.canceridc.dev/data/downloading-data/dicomweb-access)). For the purposes of this colab we have mirrored the required imaging in a Google DICOM store.

DICOM is the medical imaging format generated by CT scanners.
[DICOM](https://www.dicomstandard.org/) images are uniquely identified by three UIDs, Study Instance UID, Series Instance UID, and a SOP Instance UID.  The Colab retrieve a CT imaging from IDC by downloading all of the images associated with a CT scan. Conceptually, a Study Instance UID can be thought of as the UID that identifies all imaging acquired or generated as a result of a patient exam. Each medical image acquired as part of the exam, (e.g., CT acquisition), is identified by a unique Series Instance UID. Each image acquired or generated as part of the an acquisition is, in turn, identified with a unique SOP Instance UID.

CT imaging is commonly represented as an ordered collection of 2D images (slices). Each slice is represented by a 2D image that describes an axial cross section (volume) of the imaged body.



In [ ]:
# @title Read metadata for the DICOM instances in a series.
from typing import Sequence

import google.auth
import google.auth.transport
import pydicom
import requests


def get_series_dicomweb_url() -> str:
  return f"https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/{study_instance_uid}/series/{series_instance_uid}"

def get_sop_instance_dicomweb_urls(study_instance_uid: str, series_instance_uid: str) -> Sequence[str]:
  # Read DICOM instance metadata for imaging
  # Get oauth credentials to
  auth_credentials = google.auth.default(['https://www.googleapis.com/auth/cloud-healthcare'])[0]
  auth_credentials.refresh(google.auth.transport.requests.Request())
  headers = {"Authorization" : f"Bearer {auth_credentials.token}"}
  series = get_series_dicomweb_url(study_instance_uid, series_instance_uid)
  metadata = requests.get(f"{series}/instances", headers=headers).json()
  dicom_instances = [pydicom.Dataset.from_json(i) for i in metadata]
  # Order instance metadata by instance number
  dicom_instances = sorted(dicom_instances, key=lambda i: int(i.InstanceNumber))

  # Sample 85 slices equallly across CT volume
  MAX_SLICE = 85
  if len(dicom_instances) > MAX_SLICE:
    dicom_instances = [dicom_instances[int(round(i /(MAX_SLICE - 1) * (len(dicom_instances)-1)))] for i in range(MAX_SLICE)]
  return [f'{series}/instances/{dcm.SOPInstanceUID}' for dcm in dicom_instances]

In [ ]:
# @title Construct MedGemma 1.5 prompt formatted as Chat Completion.

# This notebook uses imaging hosted in the Imaging Data Commons (IDC) archive
# This notebook utilizes data from The Cancer Imaging Archive (TCIA).
# https://www.cancerimagingarchive.net/collection/colorectal-liver-metastases/
# Liver CRLM - CT 1026
study_instance_uid = "1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436"
series_instance_uid = "1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220"

# @markdown This section shows how to construct chat completions style requests to the endpoint using Vertex AI [prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions).

prompt_using = 'SOPInstanceUID' # @param ['SOPInstanceUID', 'SeriesInstanceUID']

# Generate chat completion formatted prompt.

if prompt_using == 'SOPInstanceUID':
  dicom_source = get_sop_instance_dicomweb_urls(study_instance_uid, series_instance_uid)
else:
  dicom_source = get_series_dicomweb_url(study_instance_uid, series_instance_uid)

instruction = ("You are an instructor teaching medical students. You are "
               "analyzing a contiguous block of CT slices from the center of "
               "the abdomen. Please review the slices provided below "
               "carefully.")
content = [{"type": "text", "text": instruction}]
content.append({"type": "image_dicom", "image_dicom": {"dicom_source": dicom_source}})
query_text = ("\n\nBased on the visual evidence in the slices provided above, "
              "is this image a good teaching example of liver pathology? "
              "Comment on hypodense lesions or other hepatic irregularities. "
              "Do not comment on findings outside the liver. Please provide "
              "your reasoning and conclude with a 'Final Answer: yes' or "
              "'Final Answer: no'.")
content.append({"type": "text", "text": query_text})

instance = {
    "@requestFormat": "chatCompletions",
    "messages": [{"role": "user", "content": content}],
    "max_tokens": 500,
    "temperature": 0
}

In [ ]:
# @title Display MedGemma 1.5 prompt.
import json
from IPython.display import display, Markdown

txt = json.dumps(instance, indent=4, sort_keys=True)
display(Markdown(f'```json\n{txt}'))

In [ ]:
# @title Configure CoLab to call MedGemma 1.5 running in Vertex AI

# @markdown #### Prerequisites

# @markdown 1. Make sure that [billing is enabled](https://cloud.google.com/billing/docs/how-to/modify-project) for your project.

# @markdown 2. Make sure that either the Compute Engine API is enabled or that you have the [Service Usage Admin](https://cloud.google.com/iam/docs/understanding-roles#serviceusage.serviceUsageAdmin) (`roles/serviceusage.serviceUsageAdmin`) role to enable the API.

# @markdown This section sets the default Google Cloud project and enables the Compute Engine API (if not already enabled), and initializes the Vertex AI API.

import os
from google.cloud import aiplatform

Google_Cloud_Project = ""  # @param {type: "string", placeholder:"e.g. MyProject"}

# @markdown To get [online predictions](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions), you will need a MedGemma [Vertex AI Endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment) that has been deployed from Model Garden. If you have not already done so, go to the [MedGemma model card](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/medgemma) and click "Deploy options > Vertex AI" to deploy the model.

# @markdown **Note:** The examples in this notebook are intended to be used with instruction-tuned variants. Make sure to use an instruction-tuned model variant to run this notebook.

# @markdown This section gets the Vertex AI Endpoint resource that you deployed from Model Garden to use for online predictions.

# @markdown Fill in the endpoint ID and region below. You can find your deployed endpoint on the [Vertex AI online prediction page](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).


ENDPOINT_ID = ""  # @param {type: "string", placeholder:"e.g. 123456789"}
ENDPOINT_REGION = ""  # @param {type: "string", placeholder:"e.g. us-central1"}

# @markdown **Note:** The colab requires dedicated [Vertex AI endpoint](https://cloud.google.com/blog/products/ai-machine-learning/reliable-ai-with-vertex-ai-prediction-dedicated-endpoints?e=48754805).

os.environ["CLOUDSDK_CORE_PROJECT"] = Google_Cloud_Project
os.environ["GOOGLE_CLOUD_PROJECT"] = Google_Cloud_Project
os.environ["GOOGLE_CLOUD_REGION"] = ENDPOINT_REGION

# Enable the Compute Engine API, if not already.
print("Enabling Compute Engine API.")
! gcloud services enable compute.googleapis.com

# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=os.environ["GOOGLE_CLOUD_PROJECT"],
                location=os.environ["GOOGLE_CLOUD_REGION"])

endpoint = aiplatform.Endpoint(
    endpoint_name=ENDPOINT_ID,
    project=Google_Cloud_Project,
    location=ENDPOINT_REGION,
)

# Use the endpoint name to check that you are using an appropriate model variant.
# These checks are based on the default endpoint name from the Model Garden
# deployment settings.
ENDPOINT_NAME = endpoint.display_name
if "pt" in ENDPOINT_NAME:
    raise ValueError(
        "The examples in this notebook are intended to be used with "
        "instruction-tuned variants. Please use an instruction-tuned model."
    )
if "text" in ENDPOINT_NAME:
    raise ValueError(
        "You are using a text-only variant which does not support multimodal"
        " inputs. Please proceed to the 'Run inference on text only' section."
    )

In [ ]:
# @title # Call MedGemma 1.5 and return prediction

import json
from IPython.display import display, Markdown

response = endpoint.raw_predict(
    body=json.dumps(instance).encode('utf-8'), use_dedicated_endpoint=True,
    headers={'Content-Type': 'application/json'}
)
response.raise_for_status()
medgemma_response = response.json()["choices"][0]["message"]["content"]

display(Markdown(f"---\n\n**[ MedGemma ]**\n\n{medgemma_response}\n\n---"))